# Exercise 4: Wind-generated set-up and circulations in a homogeneous lake

Wind over a lake can generate a water level set-up and drive both horizontal and vertical circulations. In this exercise we study set-up and circulations in a lake with uniform density. We investigate the effects of depth variation, and consider consequences of modelling this situation in either 2DH or 3D.

The lake covers an area of 10 km by 10 km and is 5 m deep. The westerly wind blows steadily with $U_{10}$ = 10.0 m/s.

## 4.1)  Wind over lake with uniform depth (2DH)

a)	Estimate the wind generated water level set-up at the east side of the lake. To do this, derive an expression for the water level set-up $\zeta$ from the depth-averaged momentum balance in $x$-direction (west-east), considering that a balance will develop between wind stress and pressure gradient. 

\begin{equation} 
    \rho_w \frac{\partial U}{\partial t} + \rho_w U \frac{\partial U}{\partial x} + \rho_w V \frac{\partial U}{\partial y} =
    \rho_w g \frac{\partial \zeta}{\partial x} + \frac{\tau_{wind,x}}{h+\zeta} - \frac{\tau_{b,x}}{h+\zeta}
\end{equation}

Note that the wind shear stress $\tau_{wind,x}$ equals:

$ \tau_{wind,x} = \rho_{air} C_D \| U_{10} \| U_{10}$

with 

$\rho_{air}$ = 1.2 kg/m$^3$

$C_D$ = 4 x 10$^{-3}$

b)	Set the parameters to simulate this situation (using a 2DH-model) and compare the estimated and computed water level set-up.

c)	Investigate the sensitivity of the set-up for changes in depth and wind speed. For what type of lakes is the water level set-up the greatest: deep or shallow lakes? 

## 4.2)  Wind over lake with non-uniform depth (2DH)

a)	For shipping purposes, a channel is dredged through the lake (depth 10 m, width 1 km). This has been included in the model schematization used next. Run the model and investigate the water levels and flow velocities for the new situation.

b)	Why is the (depth-averaged) flow in the channel directed against the wind? Try to explain this from the balance between wind stress and pressure gradient.

c)	Estimate the magnitude of the bottom shear stress $\tau_{b,x}$ in the channel, using the formula below. Does it play a role in the balance of forces?

$\tau_{b,x} = \rho_{w} c_f \sqrt{U^2 + V^2} U$

with

$\rho_w$ = 1000 gm/m$^3$

$c_f$ = 4 x 10$^{-3}$

## 4.3) Wind over lake with uniform depth (3D)

a)	Now we study again the lake with uniform depth, but now using a 3D model. Set the parameters to simulate the same situation as in part 1 of this exercise and run the model. Describe the vertical profile of the velocity.

b)	Do the 2DH and 3D simulation give the same result for the water level set-up? Discuss the role of the bottom shear stress in the 3D simulation.

## 4.4) Wind over lake with non-uniform depth (3D)

a)	We also investigate the lake with dredged channel using a 3D model. Set the parameters to simulate the same situation as in 2DH and run the model. One figure shows a vertical cross section of the horizontal velocity. Where is the near bed current the greatest?
